In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=d460d31757826ed7f409a1f2d1706324e63501c3ea76da1b525ef78dbf101213
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [3]:
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz


In [4]:
!tar xf spark-3.4.1-bin-hadoop3.tgz


In [5]:
!pip install -q findspark

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"


In [7]:
import findspark
findspark.init()


In [8]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.streaming import StreamingContext


In [9]:
# Initialize SparkSession and SparkContext
spark = SparkSession.builder.master("local").appName("WordCount").getOrCreate()
sc = spark.sparkContext

# Initialize StreamingContext with a 10-second batch interval
ssc = StreamingContext(sc, 10)


/content/spark-3.4.1-bin-hadoop3/python/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [10]:
import os

# Create a directory for the stream files
stream_dir = "/content/input.txt"
if not os.path.exists(stream_dir):
    os.mkdir(stream_dir)

# Define the text file stream
lines = ssc.textFileStream(stream_dir)


In [11]:
lines

In [12]:
# Split each line into words and count them
words = lines.flatMap(lambda line: line.split(" "))
word_counts = words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

# Print the word counts to the console
word_counts.pprint()


In [13]:
word_counts

In [14]:
# Start the streaming computation
ssc.start()



In [15]:
import time

# Create a series of files to simulate the stream
for i in range(5):
    with open(f"/content/file{i}.txt", "w") as f:
        f.write(f"This is a streaming example file {i}.\nThis example shows streaming word count.\n")
    time.sleep(10)  # Sleep for 10 seconds to simulate real-time data


-------------------------------------------
Time: 2024-09-20 17:49:50
-------------------------------------------

-------------------------------------------
Time: 2024-09-20 17:50:00
-------------------------------------------

-------------------------------------------
Time: 2024-09-20 17:50:10
-------------------------------------------

-------------------------------------------
Time: 2024-09-20 17:50:20
-------------------------------------------

-------------------------------------------
Time: 2024-09-20 17:50:30
-------------------------------------------



In [16]:
# Wait for the computation to terminate after 60 seconds
#ssc.awaitTerminationOrTimeout(60)
ssc.stop(stopSparkContext=False, stopGraceFully=True)

-------------------------------------------
Time: 2024-09-20 17:50:40
-------------------------------------------

-------------------------------------------
Time: 2024-09-20 17:50:50
-------------------------------------------

